## Deep learning on MNIST
In this exercise we will try to apply what we learnt so far on some numpy computations, but before that, we will do some profiling to check understand the performance and the inner working of the computations that we are going to apply Numba for.

The exercise here uses the [Deep learning on MNIST tutorial form Numpy](https://numpy.org/numpy-tutorials/content/tutorial-deep-learning-on-mnist.html). For the mathmetical details, you can check out the [original tutorial](https://numpy.org/numpy-tutorials/content/tutorial-deep-learning-on-mnist.html). Or even better, do that tutorial first before trying this one. For convinence, I have downloaded it and [included in this repo for you](./03.1%20-%20tutorial-deep-learning-on-mnist.ipynb).

Just like th original tutorial, we have to use Requests, Numpy and Matplotlib. We also have to import `jit` from Numba. If you don't have Matplotlib and/or Requests installed, uncommned that following and install it:

In [ ]:
#!pip install matplotlib requests

## Load and process the MNIST dataset

This is the part that we will just copy what the tutorial did. (but using more than 1000 samples.) For detailed explanation of what is done, please see the [original tutorial](./tutorial-deep-learning-on-mnist.ipynb).

In [ ]:
import gzip
import os

import numpy as np
import requests

data_sources = {
    "training_images": "train-images-idx3-ubyte.gz",  # 60,000 training images.
    "test_images": "t10k-images-idx3-ubyte.gz",  # 10,000 test images.
    "training_labels": "train-labels-idx1-ubyte.gz",  # 60,000 training labels.
    "test_labels": "t10k-labels-idx1-ubyte.gz",  # 10,000 test labels.
}
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:10.0) Gecko/20100101 Firefox/10.0"
}
request_opts = {
    "headers": headers,
    "params": {"raw": "true"},
}

# First check if the data is stored locally; if not, then download it.

data_dir = "../_data"
os.makedirs(data_dir, exist_ok=True)

base_url = "https://github.com/rossbar/numpy-tutorial-data-mirror/blob/main/"

for fname in data_sources.values():
    fpath = os.path.join(data_dir, fname)
    if not os.path.exists(fpath):
        print("Downloading file: " + fname)
        resp = requests.get(base_url + fname, stream=True, **request_opts)
        resp.raise_for_status()  # Ensure download was succesful
        with open(fpath, "wb") as fh:
            for chunk in resp.iter_content(chunk_size=128):
                fh.write(chunk)

# Decompress the 4 files and create 4 ndarrays
                
mnist_dataset = {}

# Images
for key in ("training_images", "test_images"):
    with gzip.open(os.path.join(data_dir, data_sources[key]), "rb") as mnist_file:
        mnist_dataset[key] = np.frombuffer(
            mnist_file.read(), np.uint8, offset=16
        ).reshape(-1, 28 * 28)
# Labels
for key in ("training_labels", "test_labels"):
    with gzip.open(os.path.join(data_dir, data_sources[key]), "rb") as mnist_file:
        mnist_dataset[key] = np.frombuffer(mnist_file.read(), np.uint8, offset=8)

# Split the data into training and test sets
        
x_train, y_train, x_test, y_test = (
    mnist_dataset["training_images"],
    mnist_dataset["training_labels"],
    mnist_dataset["test_images"],
    mnist_dataset["test_labels"],
)

# Normalize the arrays by dividing them by 255

training_sample, test_sample = 5000, 5000
training_images = x_train[0:training_sample] / 255
test_images = x_test[0:test_sample] / 255

# Encode the labels

def one_hot_encoding(labels, dimension=10):
    # Define a one-hot variable for an all-zero vector
    # with 10 dimensions (number labels from 0 to 9).
    one_hot_labels = labels[..., None] == np.arange(dimension)[None]
    # Return one-hot encoded labels.
    return one_hot_labels.astype(np.float64)

training_labels = one_hot_encoding(y_train[:training_sample])
test_labels = one_hot_encoding(y_test[:test_sample])

## Build and train a small neural network

Just like the original tutorial, we build a neural network from scratch. We make some adjustment, especially breaking the training process into seperate functions, so the profilling later would be easier.

In [ ]:
# Create seed
seed = 884736743
rng = np.random.default_rng(seed)

# Define hyperparameters
learning_rate = 0.005
epochs = 20
hidden_size = 100
pixels_per_image = 784
num_labels = 10

class NN:
    def __init__(self):
        self.weights_1 = 0.2 * rng.random((pixels_per_image, hidden_size)) - 0.1
        self.weights_2 = 0.2 * rng.random((hidden_size, num_labels)) - 0.1

    # Define ReLU that returns the input if it's positive and 0 otherwise.
    def _relu(self, x):
        return (x >= 0) * x

    # Set up a derivative of the ReLU function that returns 1 for a positive input
    # and 0 otherwise.
    def _relu2deriv(self, output):
        return output >= 0
        
    def _forward_prop(self, i):
        # Forward propagation/forward pass:
        # 1. The input layer:
        #    Initialize the training image data as inputs.
        self.layer_0 = training_images[i]
        # 2. The hidden layer:
        #    Take in the training image data into the middle layer by
        #    matrix-multiplying it by randomly initialized weights.
        self.layer_1 = np.dot(self.layer_0, self.weights_1)
        # 3. Pass the hidden layer's output through the ReLU activation function.
        self.layer_1 = self._relu(self.layer_1)
        # 4. Define the dropout function for regularization.
        self.dropout_mask = rng.integers(low=0, high=2, size=self.layer_1.shape)
        # 5. Apply dropout to the hidden layer's output.
        self.layer_1 *= self.dropout_mask * 2
        # 6. The output layer:
        #    Ingest the output of the middle layer into the the final layer
        #    by matrix-multiplying it by randomly initialized weights.
        #    Produce a 10-dimension vector with 10 scores.
        self.layer_2 = np.dot(self.layer_1, self.weights_2)
        
    def _back_prop(self, i):
        # Backpropagation/backward pass:
        # 1. Measure the training error (loss function) between the actual
        #    image labels (the truth) and the prediction by the model.
        self.training_loss += np.sum((training_labels[i] - self.layer_2) ** 2)
        # 2. Increment the accurate prediction count.
        self.training_accurate_predictions += int(
            np.argmax(self.layer_2) == np.argmax(training_labels[i])
        )
        # 3. Differentiate the loss function/error.
        layer_2_delta = training_labels[i] - self.layer_2
        # 4. Propagate the gradients of the loss function back through the hidden layer.
        layer_1_delta = np.dot(self.weights_2, layer_2_delta) * self._relu2deriv(self.layer_1)
        # 5. Apply the dropout to the gradients.
        layer_1_delta *= self.dropout_mask
        # 6. Update the weights for the middle and input layers
        #    by multiplying them by the learning rate and the gradients.
        self.weights_1 += learning_rate * np.outer(self.layer_0, layer_1_delta)
        self.weights_2 += learning_rate * np.outer(self.layer_1, layer_2_delta)
        
    def _training_step(self):
        # Set the initial loss/error and the number of accurate predictions to zero.
        self.training_loss = 0.0
        self.training_accurate_predictions = 0

        # For all images in the training set, perform a forward pass
        # and backpropagation and adjust the weights accordingly.
        for i in range(len(training_images)):
            self._forward_prop(i)
            self._back_prop(i)

        # Store training set losses and accurate predictions.
        self.store_training_loss.append(self.training_loss)
        self.store_training_accurate_pred.append(self.training_accurate_predictions)
        
    def _evaluation_step(self):
        results = self._relu(test_images @ self.weights_1) @ self.weights_2

        # Measure the error between the actual label (truth) and prediction values.
        test_loss = np.sum((test_labels - results) ** 2)

        # Measure prediction accuracy on test set
        test_accurate_predictions = np.sum(
            np.argmax(results, axis=1) == np.argmax(test_labels, axis=1)
        )

        # Store test set losses and accurate predictions.
        self.store_test_loss.append(test_loss)
        self.store_test_accurate_pred.append(test_accurate_predictions)
        
    def train(self, show_epochs=True):

        # To store training and test set losses and accurate predictions
        # for visualization.
        self.store_training_loss = []
        self.store_training_accurate_pred = []
        self.store_test_loss = []
        self.store_test_accurate_pred = []

        # This is a training loop.
        # Run the learning experiment for a defined number of epochs (iterations).
        for j in range(epochs):

            #################
            # Training step #
            #################

            self._training_step()

            ###################
            # Evaluation step #
            ###################

            # Evaluate model performance on the test set at each epoch.

            # Unlike the training step, the weights are not modified for each image
            # (or batch). Therefore the model can be applied to the test images in a
            # vectorized manner, eliminating the need to loop over each image
            # individually:

            self._evaluation_step()

            # Summarize error and accuracy metrics at each epoch
            if show_epochs:
                print(
                    (
                        f"Epoch: {j}\n"
                        f"  Training set error: {store_training_loss[-1] / len(training_images):.3f}\n"
                        f"  Training set accuracy: {store_training_accurate_pred[-1] / len(training_images)}\n"
                        f"  Test set error: {store_test_loss[-1] / len(test_images):.3f}\n"
                        f"  Test set accuracy: {store_test_accurate_pred[-1] / len(test_images)}"
                    )
                )

## Profile the training process

Now we use [cProfile](https://docs.python.org/3/library/profile.html) to profile the training process. The code below will take a while to run (~1 min) if you want to test it out at first, you can selected less sample in the data just like the original tutorial does.

After running the profiling process, can you identify which process in the `train()` function take the most time? Can you identify which Numpy process has been called the most? With this information, can you formulate a plan of which part of the process we can compile with jit in Numba?

In [ ]:
import cProfile, pstats
profiler = cProfile.Profile()
model = NN()
profiler.enable()
model.train(show_epochs=False)
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

Now, let's use jit to speed up some of the process. Remember what we have learnt so far. Use the tricks that we learnt and that may require rewriting of some code.

In [ ]:
from numba import jit, njit

#### TODO: use @jit or @njit to speed up the training process

class NN_jit:
    def __init__(self):
        self.weights_1 = 0.2 * rng.random((pixels_per_image, hidden_size)) - 0.1
        self.weights_2 = 0.2 * rng.random((hidden_size, num_labels)) - 0.1
        
    def reset(self):
        # extra helper function to reset the training
        self.weights_1 = 0.2 * rng.random((pixels_per_image, hidden_size)) - 0.1
        self.weights_2 = 0.2 * rng.random((hidden_size, num_labels)) - 0.1

    # Define ReLU that returns the input if it's positive and 0 otherwise.
    def _relu(self, x):
        return (x >= 0) * x

    # Set up a derivative of the ReLU function that returns 1 for a positive input
    # and 0 otherwise.
    def _relu2deriv(self, output):
        return output >= 0

    def _forward_prop(self, i):
        # Forward propagation/forward pass:
        # 1. The input layer:
        #    Initialize the training image data as inputs.
        self.layer_0 = training_images[i]
        # 2. The hidden layer:
        #    Take in the training image data into the middle layer by
        #    matrix-multiplying it by randomly initialized weights.
        self.layer_1 = np.dot(self.layer_0, self.weights_1)
        # 3. Pass the hidden layer's output through the ReLU activation function.
        self.layer_1 = self._relu(self.layer_1)
        # 4. Define the dropout function for regularization.
        self.dropout_mask = rng.integers(low=0, high=2, size=self.layer_1.shape)
        # 5. Apply dropout to the hidden layer's output.
        self.layer_1 *= self.dropout_mask * 2
        # 6. The output layer:
        #    Ingest the output of the middle layer into the the final layer
        #    by matrix-multiplying it by randomly initialized weights.
        #    Produce a 10-dimension vector with 10 scores.
        self.layer_2 = np.dot(self.layer_1, self.weights_2)

    def _back_prop(self, i):
        # Backpropagation/backward pass:
        # 1. Measure the training error (loss function) between the actual
        #    image labels (the truth) and the prediction by the model.
        self.training_loss += np.sum((training_labels[i] - self.layer_2) ** 2)
        # 2. Increment the accurate prediction count.
        self.training_accurate_predictions += int(
            np.argmax(self.layer_2) == np.argmax(training_labels[i])
        )
        # 3. Differentiate the loss function/error.
        layer_2_delta = training_labels[i] - self.layer_2
        # 4. Propagate the gradients of the loss function back through the hidden layer.
        layer_1_delta = np.dot(self.weights_2, layer_2_delta) * self._relu2deriv(self.layer_1)
        # 5. Apply the dropout to the gradients.
        layer_1_delta *= self.dropout_mask
        # 6. Update the weights for the middle and input layers
        #    by multiplying them by the learning rate and the gradients.
        self.weights_1 += learning_rate * np.outer(self.layer_0, layer_1_delta)
        self.weights_2 += learning_rate * np.outer(self.layer_1, layer_2_delta)
        
    def _training_step(self):
        # Set the initial loss/error and the number of accurate predictions to zero.
        self.training_loss = 0.0
        self.training_accurate_predictions = 0

        # For all images in the training set, perform a forward pass
        # and backpropagation and adjust the weights accordingly.
        for i in range(len(training_images)):
            self._forward_prop(i)
            self._back_prop(i)

        # Store training set losses and accurate predictions.
        self.store_training_loss.append(self.training_loss)
        self.store_training_accurate_pred.append(self.training_accurate_predictions)
        
    def _evaluation_step(self):
        results = self._relu(test_images @ self.weights_1) @ self.weights_2

        # Measure the error between the actual label (truth) and prediction values.
        test_loss = np.sum((test_labels - results) ** 2)

        # Measure prediction accuracy on test set
        test_accurate_predictions = np.sum(
            np.argmax(results, axis=1) == np.argmax(test_labels, axis=1)
        )

        # Store test set losses and accurate predictions.
        self.store_test_loss.append(test_loss)
        self.store_test_accurate_pred.append(test_accurate_predictions)
        
    def train(self, show_epochs=True):

        # To store training and test set losses and accurate predictions
        # for visualization.
        self.store_training_loss = []
        self.store_training_accurate_pred = []
        self.store_test_loss = []
        self.store_test_accurate_pred = []

        # This is a training loop.
        # Run the learning experiment for a defined number of epochs (iterations).
        for j in range(epochs):

            #################
            # Training step #
            #################

            self._training_step()

            ###################
            # Evaluation step #
            ###################

            # Evaluate model performance on the test set at each epoch.

            # Unlike the training step, the weights are not modified for each image
            # (or batch). Therefore the model can be applied to the test images in a
            # vectorized manner, eliminating the need to loop over each image
            # individually:

            self._evaluation_step()

            # Summarize error and accuracy metrics at each epoch
            if show_epochs:
                print(
                    (
                        f"Epoch: {j}\n"
                        f"  Training set error: {store_training_loss[-1] / len(training_images):.3f}\n"
                        f"  Training set accuracy: {store_training_accurate_pred[-1] / len(training_images)}\n"
                        f"  Test set error: {store_test_loss[-1] / len(test_images):.3f}\n"
                        f"  Test set accuracy: {store_test_accurate_pred[-1] / len(test_images)}"
                    )
                )

Profile again, remember to run it once to compile first

In [ ]:
# Compile the process

model_jit = NN_jit()
model_jit.train(show_epochs=False) # or just run the jit'ed methods

In [ ]:
profiler = cProfile.Profile()
model_jit.reset()
model_jit.train(show_epochs=False)
profiler.enable()
model_jit.train(show_epochs=False)
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

Did you managed to get some performance improvement? Have you try with `@njit` or `@jit(nopython=True)`? Challange yourself by using no Python mode and rewrite some of the code. See if that would improve the performance further. 

## Plot the result

This part has nothing to do with the profilling and compiling for speed up but just to check the training result.

In [ ]:
import matplotlib.pyplot as plt

# The training set metrics.
y_training_error = [
    model.store_training_loss[i] / float(len(training_images))
    for i in range(len(model.store_training_loss))
]
x_training_error = range(1, len(model.store_training_loss) + 1)
y_training_accuracy = [
    model.store_training_accurate_pred[i] / float(len(training_images))
    for i in range(len(model.store_training_accurate_pred))
]
x_training_accuracy = range(1, len(model.store_training_accurate_pred) + 1)

# The test set metrics.
y_test_error = [
    model.store_test_loss[i] / float(len(test_images)) for i in range(len(model.store_test_loss))
]
x_test_error = range(1, len(model.store_test_loss) + 1)
y_test_accuracy = [
    model.store_training_accurate_pred[i] / float(len(training_images))
    for i in range(len(model.store_training_accurate_pred))
]
x_test_accuracy = range(1, len(model.store_test_accurate_pred) + 1)

# Display the plots.
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
axes[0].set_title("Training set error, accuracy")
axes[0].plot(x_training_accuracy, y_training_accuracy, label="Training set accuracy")
axes[0].plot(x_training_error, y_training_error, label="Training set error")
axes[0].set_xlabel("Epochs")
axes[1].set_title("Test set error, accuracy")
axes[1].plot(x_test_accuracy, y_test_accuracy, label="Test set accuracy")
axes[1].plot(x_test_error, y_test_error, label="Test set error")
axes[1].set_xlabel("Epochs")
plt.show()